In [31]:
import pandas as pd

# Memuat data alamat
alamat_df = pd.read_excel('Data Alamat.xlsx', sheet_name='SMT')
print(alamat_df.head())
print(f"Baris: {len(alamat_df)} | Kolom: {list(alamat_df.columns)}")


                                              ALAMAT           kecamatan  \
0  SRINANTI RT/RW 005/003 DESA SUNGAI GERONG KEC ...    KEC. BANYUASIN.I   
1  JL. KELAPA LK IV KEL. SUKA MAJU KEC. BINJAI BA...  KEC. BINJAI BARAT    
2  UJUNG PADANG RT/RW -/- KEL. SASAK KEC. SASAK R...                 NaN   
3  DESA ONOWAEMBO KEC. GUNUNGSITOLI IDANOI KOTA G...                 NaN   
4  JL. HUTA DOLOK III RT/RW 000/000 KEL. LBN DOLO...                 NaN   

           kabupaten        area_polreg Province_polreg  
0          BANYUASIN          BANYUASIN  South Sumatera  
1             BINJAI             Binjai  North Sumatera  
2  KAB PASAMAN BARAT  KAB PASAMAN BARAT   West Sumatera  
3      GUNUNG SITOLI               Nias  North Sumatera  
4               TOBA            Samosir  North Sumatera  
Baris: 47815 | Kolom: ['ALAMAT', 'kecamatan', 'kabupaten', 'area_polreg', 'Province_polreg']


In [32]:
# Memuat data referensi kecamatan (BPS 2017)
kecamatan_df = pd.read_excel('Daftar Kecamatan Indonesia BPS 2017.xlsx')
print(kecamatan_df.head())
print(f"Baris: {len(kecamatan_df)} | Kolom: {list(kecamatan_df.columns)}")

              Kecamatan  Kecamatan_ID           Kabupaten  Kabupaten_ID  \
0  2 X 11 ENAM LINGKUNG            50     PADANG PARIAMAN             6   
1                  ABUN            30            TAMBRAUW             9   
2                  ABOY            41  PEGUNUNGAN BINTANG            17   
3         ABUNG  KUNANG            32       LAMPUNG UTARA             6   
4         ABUNG SELATAN            50       LAMPUNG UTARA             6   

         Provinsi  Provinsi_ID  
0  SUMATERA BARAT           13  
1     PAPUA BARAT           91  
2           PAPUA           94  
3         LAMPUNG           18  
4         LAMPUNG           18  
Baris: 6678 | Kolom: ['Kecamatan', 'Kecamatan_ID', 'Kabupaten', 'Kabupaten_ID', 'Provinsi', 'Provinsi_ID']


In [33]:
import re
from rapidfuzz import process, fuzz

STOP_TOKENS = {
    'KABUPATEN', 'KAB', 'KOTA', 'KECAMATAN', 'KEC', 'PROVINSI', 'PROV',
    'KELURAHAN', 'KEL', 'DESA', 'DS', 'RT', 'RW', 'JL', 'JLN', 'JALAN',
    'LINGKUNGAN', 'LK', 'NO', 'NOMOR', 'GANG', 'GG', 'LAINNYA', 'PS', 'PASAR',
    'BLOK', 'DUSUN', 'DSN', 'KOMP', 'COMPLEX'
}

ABBREV_MAP = {
    'KAB.': 'KABUPATEN',
    'KAB ': 'KABUPATEN ',
    'KAB-': 'KABUPATEN ',
    'KOTA ': 'KOTA ',
    'KOT.': 'KOTA',
    'KOTAAdm': 'KOTA',
    'KEC.': 'KECAMATAN',
    'KEC ': 'KECAMATAN ',
    'KCAMATAN': 'KECAMATAN',
    'KCAM.': 'KECAMATAN',
    'KCM.': 'KECAMATAN',
    'KEM.': 'KECAMATAN',
    'D.K.': 'DKI',
}

PUNCT_PATTERN = re.compile(r"[.,/\\-]")
MULTISPACE_PATTERN = re.compile(r"\s+")


def normalize_text(text: str) -> str:
    text = text.upper()
    for src, dst in ABBREV_MAP.items():
        text = text.replace(src, dst)
    text = PUNCT_PATTERN.sub(' ', text)
    text = MULTISPACE_PATTERN.sub(' ', text)
    return text.strip()


def clean_candidate(candidate: str) -> str:
    tokens = []
    for token in candidate.strip().split():
        if token in STOP_TOKENS:
            break
        tokens.append(token)
    return ' '.join(tokens).strip()


def extract_keyword_segment(text: str, keyword: str) -> str:
    keyword = keyword.upper()
    if keyword not in text:
        return ''
    start = text.index(keyword) + len(keyword)
    tail = text[start:]
    # Berhenti ketika menemukan token kata kunci lain
    split = re.split(r"\b(KABUPATEN|KOTA|KECAMATAN|PROVINSI|KELURAHAN|DESA|RT|RW|JL|JALAN|LINGKUNGAN|LK)\b", tail, maxsplit=1)
    candidate = split[0] if split else tail
    return clean_candidate(candidate)


def fuzzy_match(name: str, choices, scorer=fuzz.WRatio, score_cutoff=85, processor=None):
    if not name:
        return ''
    match = process.extractOne(
        name,
        choices,
        scorer=scorer,
        score_cutoff=score_cutoff,
        processor=processor,
    )
    return match[0] if match else ''


In [34]:
# Prepare normalized reference data
kecamatan_df['Kecamatan_norm'] = kecamatan_df['Kecamatan'].apply(normalize_text)
kabupaten_df = kecamatan_df[['Kabupaten']].drop_duplicates().copy()
kabupaten_df['Kabupaten_norm'] = kabupaten_df['Kabupaten'].apply(normalize_text)

kecamatan_choices = list(dict.fromkeys(kecamatan_df['Kecamatan']))
kabupaten_choices = list(dict.fromkeys(kabupaten_df['Kabupaten']))

kecamatan_norm_map = {}
for original, norm in zip(kecamatan_df['Kecamatan'], kecamatan_df['Kecamatan_norm']):
    kecamatan_norm_map.setdefault(norm, original)

kabupaten_norm_map = {}
for original, norm in zip(kabupaten_df['Kabupaten'], kabupaten_df['Kabupaten_norm']):
    kabupaten_norm_map.setdefault(norm, original)


def contains_reference(text: str, norm_map: dict) -> str:
    for norm_name, original in norm_map.items():
        if norm_name and norm_name in text:
            return original
    return ''


def resolve_kecamatan(text: str) -> str:
    candidate = extract_keyword_segment(text, 'KECAMATAN')
    if candidate:
        candidate_norm = normalize_text(candidate)
        if candidate_norm in kecamatan_norm_map:
            return kecamatan_norm_map[candidate_norm]
        match = fuzzy_match(candidate, kecamatan_choices, processor=normalize_text)
        if match:
            return match
    direct = contains_reference(text, kecamatan_norm_map)
    if direct:
        return direct
    match = fuzzy_match(text, kecamatan_choices, processor=normalize_text, score_cutoff=90)
    return match or ''


def resolve_kabupaten(text: str) -> str:
    for keyword in ('KABUPATEN', 'KOTA'):
        candidate = extract_keyword_segment(text, keyword)
        if candidate:
            candidate_norm = normalize_text(candidate)
            if candidate_norm in kabupaten_norm_map:
                return kabupaten_norm_map[candidate_norm]
            match = fuzzy_match(candidate, kabupaten_choices, processor=normalize_text)
            if match:
                return match
    direct = contains_reference(text, kabupaten_norm_map)
    if direct:
        return direct
    match = fuzzy_match(text, kabupaten_choices, processor=normalize_text, score_cutoff=88)
    return match or ''


In [35]:
import numpy as np


def is_nullish(value) -> bool:
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return True
    if isinstance(value, str):
        stripped = value.strip()
        return stripped == '' or stripped.upper() == 'NULL'
    return False


alamat_df['alamat_norm'] = alamat_df['ALAMAT'].astype(str).apply(normalize_text)
needs_kec = alamat_df['kecamatan'].apply(is_nullish)
needs_kab = alamat_df['kabupaten'].apply(is_nullish)

updated_kec = alamat_df['kecamatan'].astype(object).copy()
updated_kab = alamat_df['kabupaten'].astype(object).copy()

for idx, row in alamat_df.loc[needs_kec | needs_kab, ['alamat_norm']].itertuples():
    text = row
    if needs_kec.at[idx]:
        resolved_kec = resolve_kecamatan(text)
        if resolved_kec:
            updated_kec.at[idx] = resolved_kec
    if needs_kab.at[idx]:
        resolved_kab = resolve_kabupaten(text)
        if resolved_kab:
            updated_kab.at[idx] = resolved_kab

alamat_df['kecamatan'] = updated_kec
alamat_df['kabupaten'] = updated_kab

still_missing_kec = alamat_df.loc[alamat_df['kecamatan'].apply(is_nullish), 'kecamatan'].shape[0]
still_missing_kab = alamat_df.loc[alamat_df['kabupaten'].apply(is_nullish), 'kabupaten'].shape[0]

filled_kec = (needs_kec & alamat_df['kecamatan'].apply(lambda x: not is_nullish(x))).sum()
filled_kab = (needs_kab & alamat_df['kabupaten'].apply(lambda x: not is_nullish(x))).sum()

print(f"Kecamatan newly filled: {filled_kec} of {needs_kec.sum()} originally missing")
print(f"Kabupaten newly filled: {filled_kab} of {needs_kab.sum()} originally missing")
print(f"Remaining NULL kecamatan: {still_missing_kec}")
print(f"Remaining NULL kabupaten: {still_missing_kab}")

alamat_df.drop(columns=['alamat_norm'], inplace=True)


Kecamatan newly filled: 24214 of 24233 originally missing
Kabupaten newly filled: 0 of 0 originally missing
Remaining NULL kecamatan: 19
Remaining NULL kabupaten: 0


In [36]:
remaining_rows = alamat_df[alamat_df['kecamatan'].apply(is_nullish) | alamat_df['kabupaten'].apply(is_nullish)]
print(f"Baris yang masih kosong kecamatan/kabupaten: {len(remaining_rows)}")
remaining_rows[['ALAMAT', 'kecamatan', 'kabupaten']].head(10)

Baris yang masih kosong kecamatan/kabupaten: 19


,ALAMAT,kecamatan,kabupaten
4367,DUSUN I RAMBAI STM HILIR KAB. DELI SERDANG,NaN,DELI SERDANG
8160,DUSUN SANGGA PURA BELINTEN SEI BINGEI KABUPATE...,NaN,LANGKAT
10202,DUSUN II RUMAH LENGO STM HULU KABUPATEN DELI S...,NaN,DELI SERDANG
12603,JL PERWIRA I NO 33 KOTA MEDAN,NaN,MEDAN
12604,JL PERWIRA I NO 33 KOTA MEDAN,NaN,MEDAN
13244,DUSUN V KP DALAM TADUKAN RAGA STM HILIR KABUPA...,NaN,DELI SERDANG
18864,"JALAN MH THAMRIN NO. 67,67-A,67-B KOTA MEDAN",NaN,MEDAN
20207,DUSUN V GEMINI POINT TADUKAN RAGA STM HILIR KA...,NaN,DELI SERDANG
26815,DOLAT RAKYAT DOLAT RAKYAT DOLAT RAKYAT KABUPAT...,NaN,TANAH KARO
30041,DUSUN ADI MULIO HILIR EMPLASMEN KWALA MENCIRIM...,NaN,LANGKAT


In [37]:
# Simpan pembaruan kembali ke sheet SMT (tutup file Excel sebelum menjalankan cell ini)
output_path = 'Data Alamat.xlsx'
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    alamat_df.to_excel(writer, sheet_name='SMT', index=False)

print('Sheet SMT yang diperbarui telah ditulis ke Data Alamat.xlsx')

Sheet SMT yang diperbarui telah ditulis ke Data Alamat.xlsx
